# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

Importing everything required

In [1]:
from configparser import ConfigParser
from os import environ

from pandas import to_timedelta, Timestamp, read_csv
from pyspark.sql import SparkSession
from pyspark.sql.types import DateType
from pyspark.sql.functions import udf, col, lit, year, month, upper, to_date, monotonically_increasing_id, count

Setting up the configurations from the config file

In [2]:
config = ConfigParser()
config.read('AWS.cfg', encoding='utf-8-sig')

environ['AWS_ACCESS_KEY_ID'] = config['AWS']['AWS_ACCESS_KEY_ID']
environ['AWS_SECRET_ACCESS_KEY'] = config['AWS']['AWS_SECRET_ACCESS_KEY']
output_data_path = config['S3']['DEST_S3_BUCKET']

Creating a Spark Session

In [3]:
spark = SparkSession.builder \
    .config("spark.jars.repositories", "https://repos.spark-packages.org/") \
    .config("spark.jars.packages", "saurfang:spark-sas7bdat:2.0.0-s_2.11,org.apache.hadoop:hadoop-aws:2.7.0") \
    .enableHiveSupport().getOrCreate()

Creating a user defined function to parse the date in correct format

In [4]:
convert_to_datetime = udf(lambda date: to_timedelta(date, unit='D') + Timestamp('1960-1-1') if date else None, DateType())

## US National Tourism and Trade Office Dataset

Reading the data in spark

In [5]:
immigration_data_path = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
df = spark.read.format('com.github.saurfang.sas.spark').load(immigration_data_path)
df.createOrReplaceTempView('immigration_data')

#### IMMIGRATION_INFO Table

In [6]:
# extracting the required columns

immigration_info = spark.sql("""
        SELECT DISTINCT cicid AS cic_id,
                        i94yr AS year, 
                        i94mon  AS month, 
                        i94port AS city_code,
                        i94addr AS state_code,
                        arrdate AS arrive_date, 
                        depdate  AS departure_date, 
                        i94mode AS mode,
                        i94visa AS visa
        FROM immigration_data
    """)
# add the primary key
immigration_info = immigration_info.withColumn("immigration_id", monotonically_increasing_id())

# adding country info for readability
immigration_info = immigration_info.withColumn('country', lit('United States'))

# changing the date type columns to have datetime values
immigration_info = immigration_info.withColumn('arrive_date', convert_to_datetime(col('arrive_date')))
immigration_info = immigration_info.withColumn('departure_date', convert_to_datetime(col('departure_date')))

#### IMMIGRATION_PERSONAL Table

In [7]:
# extracting the required columns
immigration_personal = spark.sql("""
            SELECT  DISTINCT cicid AS cic_id, 
                             i94cit AS citizen_country, 
                             i94res  AS residence_country, 
                             biryear AS birth_year,
                             gender,
                             insnum AS ins_num
            FROM immigration_data
        """)
# add the primary key
immigration_personal = immigration_personal.withColumn("personal_id", monotonically_increasing_id())

#### IMMIGRATION_AIRLINE Table

In [8]:
# extracting the required columns

immigration_airline = spark.sql("""
                SELECT  DISTINCT cicid AS cic_id, 
                                 airline, 
                                 admnum  AS admin_num, 
                                 fltno AS flight_number,
                                 visatype AS visa_type
                FROM immigration_data
            """)
# add the primary key
immigration_airline = immigration_airline.withColumn("airline_id", monotonically_increasing_id())

### I94 SAS labels desctiption

We can examine the contents of the file by opening it in any editor. The magic numbers we see in the code have been obtained by carefull examination of the file.

In [9]:
# reading the file

with open("I94_SAS_Labels_Descriptions.SAS") as f:
    contents = f.readlines()

#### COUNTRY_CODE Table

In [10]:
# Extracting country codes from labes data. The magic numbers here have ben chosen after inspecting the file manually
country_code = {}
for countries in contents[10:298]:
    pair = countries.split('=')
    code, country = pair[0].strip(), pair[1].strip().strip("'")
    country_code[code] = country
df_country_codes = spark.createDataFrame(country_code.items(), ['code', 'country'])

In [11]:
df_country_codes.head(5)

[Row(code='236', country='AFGHANISTAN'),
 Row(code='101', country='ALBANIA'),
 Row(code='316', country='ALGERIA'),
 Row(code='102', country='ANDORRA'),
 Row(code='324', country='ANGOLA')]

#### CITY_CODE Table

In [13]:
# Extracting city codes from labes data. The magic numbers here have ben chosen after inspecting the file manually
city_code = {}
for cities in contents[303:962]:
    pair = cities.split('=')
    code, city = pair[0].strip("\t").strip().strip("'"),  pair[1].strip('\t').strip().strip("''")
    city_code[code] = city
df_city_codes = spark.createDataFrame(city_code.items(), ['code', 'city'])

In [14]:
df_city_codes.show()

+----+--------------------+
|code|                city|
+----+--------------------+
| ANC|ANCHORAGE, AK    ...|
| BAR|BAKER AAF - BAKER...|
| DAC|DALTONS CACHE, AK...|
| PIZ|DEW STATION PT LA...|
| DTH|DUTCH HARBOR, AK ...|
| EGL|EAGLE, AK        ...|
| FRB|FAIRBANKS, AK    ...|
| HOM|HOMER, AK        ...|
| HYD|HYDER, AK        ...|
| JUN|JUNEAU, AK       ...|
| 5KE|       KETCHIKAN, AK|
| KET|KETCHIKAN, AK    ...|
| MOS|MOSES POINT INTER...|
| NIK|NIKISKI, AK      ...|
| NOM|NOM, AK          ...|
| PKC|POKER CREEK, AK  ...|
| ORI|  PORT LIONS SPB, AK|
| SKA|SKAGWAY, AK      ...|
| SNP| ST. PAUL ISLAND, AK|
| TKI|          TOKEEN, AK|
+----+--------------------+
only showing top 20 rows



#### STATE_CODE Table

In [15]:
# Extracting state codes from labes data. The magic numbers here have ben chosen after inspecting the file manually
state_code = {}
for states in contents[982:1036]:
    pair = states.split('=')
    code, state = pair[0].strip('\t').strip("'"), pair[1].strip().strip("'")
    state_code[code] = state

df_state_codes = spark.createDataFrame(state_code.items(), ['code', 'state'])

In [16]:
df_state_codes.show()

+----+-----------------+
|code|            state|
+----+-----------------+
|  AK|           ALASKA|
|  AZ|          ARIZONA|
|  AR|         ARKANSAS|
|  CA|       CALIFORNIA|
|  CO|         COLORADO|
|  CT|      CONNECTICUT|
|  DE|         DELAWARE|
|  DC|DIST. OF COLUMBIA|
|  FL|          FLORIDA|
|  GA|          GEORGIA|
|  GU|             GUAM|
|  HI|           HAWAII|
|  ID|            IDAHO|
|  IL|         ILLINOIS|
|  IN|          INDIANA|
|  IA|             IOWA|
|  KS|           KANSAS|
|  KY|         KENTUCKY|
|  LA|        LOUISIANA|
|  ME|            MAINE|
+----+-----------------+
only showing top 20 rows



## World Temperature Dataset

In [17]:
# reading the data
df = spark.read.csv('../../data2/GlobalLandTemperaturesByCity.csv', header=True)
df.createOrReplaceTempView("temp_data")

#### TEMPERATURE Table

In [18]:
# extarcting the required columns
temperature = spark.sql("""
    SELECT DISTINCT dt, 
                    AverageTemperature AS avg_temp, 
                    AverageTemperatureUncertainty  AS avg_temp_uncertainity, 
                    City AS city,
                    Country AS country
    FROM temp_data
    WHERE country == 'United States'
""")

# add the primary key
temperature = temperature.withColumn("temperature_id", monotonically_increasing_id())
# format the columns
temperature = temperature.withColumn('dt', to_date(col('dt')))
temperature = temperature.withColumn('year', year(temperature['dt']))
temperature = temperature.withColumn('month', month(temperature['dt']))

## US city demogrphics dataset

In [19]:
# reading the data

df = spark.read.format('csv').options(header=True, delimiter=';').load('us-cities-demographics.csv')
df.createOrReplaceTempView('demographics')

#### DEMOGRAPHICS_POPULATION Table

In [20]:
# extarcting the required columns

demographics = spark.sql("""
        SELECT DISTINCT UPPER(City) AS city, 
                        UPPER(State) AS state, 
                        "Median Age"  AS median_age, 
                        "Average Household Size" AS avg_household_size, 
                        "Male Population"  AS male_population, 
                        "Female Population" AS female_population,
                        "Number of Veterans" AS num_vetarans,
                        "Foreign-born" AS foreign_born,
                        Race AS race
        FROM demographics
    """)
# add the primary key
demographics = demographics.withColumn("demographic_id", monotonically_increasing_id())

# Data Quality Checks

We can perform null check to make sure that the data is correctly inserted into the tables and no tables are empty

In [21]:
strs = ['immigration_info', 'immigration_personal', 'immigration_airline',
        'temperature', 'demographics', 'country_codes', 'city_codes', 'state_codes']
tables = [immigration_info, immigration_personal, immigration_airline, temperature,
          demographics, df_country_codes, df_city_codes, df_state_codes]

for df, name in zip(tables, strs):
    print(f'Table: {name}')
    record_num = df.count()
    if record_num == 0:
        raise ValueError(f"Records are not inserted properly. Table {name} is empty")
    else:
        print(f'Data quality check passed for {name}. Rows found: {record_num}')
    df.printSchema()

Table: immigration_info
Data quality check passed for immigration_info. Rows found: 3096313
root
 |-- cic_id: double (nullable = true)
 |-- year: double (nullable = true)
 |-- month: double (nullable = true)
 |-- city_code: string (nullable = true)
 |-- state_code: string (nullable = true)
 |-- arrive_date: date (nullable = true)
 |-- departure_date: date (nullable = true)
 |-- mode: double (nullable = true)
 |-- visa: double (nullable = true)
 |-- immigration_id: long (nullable = false)
 |-- country: string (nullable = false)

Table: immigration_personal
Data quality check passed for immigration_personal. Rows found: 3096313
root
 |-- cic_id: double (nullable = true)
 |-- citizen_country: double (nullable = true)
 |-- residence_country: double (nullable = true)
 |-- birth_year: double (nullable = true)
 |-- gender: string (nullable = true)
 |-- ins_num: string (nullable = true)
 |-- personal_id: long (nullable = false)

Table: immigration_airline
Data quality check passed for immigrat

We can perform integrity/type check to make sure that all the dates are in datetime format

In [29]:
if type(immigration_info.schema['arrive_date'].dataType) == DateType and \
        type(immigration_info.schema['departure_date'].dataType) == DateType:

    print(f'Integrity check passed for immigration_info table.')

else:
    raise ValueError(f"Records are not inserted in the right format in immigration_info table.")

Integrity check passed for immigration_info table.


# Performing Analytics

* How many people from a specific country visit the city of new york in a given year?

In [22]:
df_country_codes[df_country_codes['country'] == 'PAKISTAN'].show()

+----+--------+
|code| country|
+----+--------+
| 258|PAKISTAN|
+----+--------+



In [23]:
df_city_codes[df_city_codes['city'].contains('NEW YORK')].show()

+----+--------------------+
|code|                city|
+----+--------------------+
| NYC|NEW YORK, NY     ...|
+----+--------------------+



In [24]:
joined_immigration = immigration_info.join(immigration_personal, immigration_info.cic_id == immigration_personal.cic_id)

In [25]:
joined_immigration.where(joined_immigration.city_code == 'NYC').where(joined_immigration.citizen_country == 258).groupby("year").count().show()

+------+-----+
|  year|count|
+------+-----+
|2016.0| 1380|
+------+-----+



* How many people from a specific country have visited every city?

In [26]:
joined_immigration.join(df_city_codes, joined_immigration.city_code == df_city_codes.code).where(joined_immigration.citizen_country == 258).groupby("city").count().show()

+--------------------+-----+
|                city|count|
+--------------------+-----+
|FORT MYERS, FL   ...|   26|
|LOS ANGELES, CA  ...|  343|
|WEST PALM BEACH, ...|   46|
|THOUSAND ISLAND B...|   14|
|NEW YORK, NY     ...| 1380|
|DETROIT, MI      ...|  142|
|AUSTIN, TX       ...|    3|
|PORT EVERGLADES, ...|    1|
|SHANNON, IRELAND ...|    1|
|FORT LAUDERDALE, ...|  110|
|ORLANDO, FL      ...|   98|
|  KAHULUI - MAUI, HI|   14|
|DEL BONITA, MT   ...|    1|
|OTTAWA, CANADA   ...|    8|
|MONTREAL, CANADA ...|   12|
|BRADENTON - SARAS...|    2|
|ST PAUL, MN      ...|   18|
|CALGARY, CANADA  ...|   14|
|CHARLOTTE, NC    ...|   20|
|  No PORT Code (YGF)|    1|
+--------------------+-----+
only showing top 20 rows



# Writing Data to S3

In [30]:
# writing immigration_info table to S3 in parquet files after partitioning by state
immigration_info.write.mode("ignore").partitionBy('state_code').parquet(output_data_path + 'immigration_info')

In [31]:
# writing immigration_personal table to S3 in parquet files
immigration_personal.write.mode("ignore").parquet(output_data_path + 'immigration_personal')

In [32]:
# writing immigration_airline table to S3 in parquet files
immigration_airline.write.mode("ignore").parquet(output_data_path + 'immigration_airline')

In [33]:
# write temperature table to parquet files
temperature.write.mode("ignore").parquet(output_data_path + 'temperature')

In [34]:
# write demographics table to parquet files
demographics.write.mode("ignore").parquet(output_data_path + 'demographics')

In [35]:
# writing state_codes table to S3 in parquet files
df_state_codes.write.mode("ignore").parquet(output_data_path + 'state_code')

In [36]:
# writing country_codes table to S3 in parquet files
df_country_codes.write.mode("ignore").parquet(output_data_path + 'country_code')

In [37]:
# writing city_codes table to S3 in parquet files
df_city_codes.write.mode("ignore").parquet(output_data_path + 'city_code')